In [ ]:
%pip install datasets --upgrade
import os

# Set HF token if needed
os.environ["HF_TOKEN"] = "hf_token"

In [3]:
from datasets import load_dataset

# Load MedQuAD dataset - using a sample for faster training
medquad_dataset = load_dataset("lavita/MedQuAD")

# Take a subset for faster training (adjust size as needed)
medquad_subset_size = 10000  
if len(medquad_dataset) > medquad_subset_size:
    medquad_dataset = medquad_dataset.shuffle(seed=42).select(range(medquad_subset_size))

print(f"✅ MedQuAD dataset loaded: {len(medquad_dataset)} samples")
print(f"Dataset columns: {medquad_dataset.column_names}")

# Show sample
print("\nSample from MedQuAD:")
sample = medquad_dataset[0]
for key, value in sample.items():
    print(f"{key}: {str(value)[:200]}...")


ConnectionError: Couldn't reach 'lavita/MedQuAD' on the Hub (LocalEntryNotFoundError)

In [ ]:
# Define prompt template
alpaca_prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def format_medquad_dataset(examples):
    """Format MedQuAD dataset to Alpaca format"""
    # Handle different possible column names in MedQuAD
    if 'question' in examples and 'answer' in examples:
        questions = examples['question']
        answers = examples['answer']
        instructions = ["Answer the following medical question accurately and comprehensively."] * len(questions)
        inputs = [""] * len(questions)
    else:
        # Try other common column names
        possible_q_cols = ['question', 'input', 'query', 'text']
        possible_a_cols = ['answer', 'output', 'response', 'target']
        
        q_col = None
        a_col = None
        
        for col in possible_q_cols:
            if col in examples:
                q_col = col
                break
        
        for col in possible_a_cols:
            if col in examples:
                a_col = col
                break
        
        if q_col and a_col:
            questions = examples[q_col]
            answers = examples[a_col]
            instructions = ["Answer the following medical question accurately and comprehensively."] * len(questions)
            inputs = [""] * len(questions)
        else:
            raise ValueError(f"Could not find question/answer columns in: {list(examples.keys())}")
    
    # Format using Alpaca template
    EOS_TOKEN = tokenizer.eos_token
    texts = [
        alpaca_prompt_template.format(instruction, inp, answer) + EOS_TOKEN
        for instruction, inp, answer in zip(instructions, inputs, answers)
    ]
    
    return {"text": texts}

# Format the dataset
print("🔄 Formatting MedQuAD dataset...")
medquad_formatted = medquad_dataset.map(format_medquad_dataset, batched=True)
print(f"✅ MedQuAD dataset formatted: {len(medquad_formatted)} samples")

# Show formatted sample
print("\nFormatted sample:")
print(medquad_formatted[0]['text'][:500] + "...")